Quick notebook to demonstrate a problem I encountered combining `docplex` and `pandas`.

In [1]:
from docplex.mp.model import Model
import  pandas as pd
m = Model()
v = m.continuous_var()
df = pd.DataFrame({"a":[1, 2, 3], "b" : [4, 5, 6], "data":[1, 12, 19]})
df.set_index(["a", "b"], inplace=True)
df2 = pd.DataFrame({"a":[1, 2, 5], "b" : [4, 5, 2], "data":[1.1, 12.1, 19]})
df2.set_index(["a", "b"], inplace=True)

Ok, set up finished, here is the buggy step.

In [2]:
df.join(df2, how="outer", rsuffix="_2").fillna(v)

DOcplexException: Model<docplex_model1> did not solve successfully

This looks like a `docplex`, `pandas` bug. I'm inclined to think it's a `docplex` bug because I can combine `.fillna` with new style classes just fine. (I.e. see below).

In [3]:
class MyClass(object):
    def __repr__(self):
        return "<MyClass>"
mc = MyClass()

In [4]:
df.join(df2, how="outer", rsuffix="_2").fillna(mc)

,,data,data_2
a,b,,
1,4,1,1.1
2,5,12,12.1
3,6,19,<MyClass>
5,2,<MyClass>,19


However, the most likely usage pattern is where I want not a variable but instead a `gurobipy` friendly proxy for zero. Luckily, that does work!

In [10]:
df.join(df2, how="outer", rsuffix="_2").fillna(0)

,,data,data_2
a,b,,
1,4,1,1.1
2,5,12,12.1
3,6,19,0.0
5,2,0,19.0


In [6]:
v

docplex.mp.linear.Var(type=C,name=_x1*)

In [7]:
v+v-v

docplex.mp.LinearExpr(_x1)